# Eat Safe, Love

## Notebook Set Up

In [36]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

In [37]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [38]:
# assign the uk_food database to a variable name
db = mongo['UK_food']

In [39]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [40]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [41]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
query = {}
results = establishments.find(query)
# Use count_documents to display the number of documents in the result
establishments.count_documents(query)
for x in results:
    pprint(x)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67dcab72f582ffc9d9e8a83b'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'Loc

In [42]:
# Convert the result to a Pandas DataFrame

hygiene = establishments.find(query)

hygiene_df = pd.DataFrame(hygiene)

# Display the number of rows in the DataFrame
# print(hygiene_df.shape[0])

# Display the first 10 rows of the DataFrame
hygiene_df.head()

,_id,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending
0,67dcab72f582ffc9d9e8a83b,Penang Flavours,Restaurant/Cafe/Canteen,1,Penang Flavours,146A Plumstead Rd,London,,SE18 7DY,,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': '', 'Structural': '', 'ConfidenceI...",FHRS,"{'longitude': 0.08384, 'latitude': 51.490142}",,4623.972328,True
1,67dcb48ce5b554d991c05224,Penang Flavours,Restaurant/Cafe/Canteen,,Penang Flavours,146A Plumstead Rd,London,,SE18 7DY,,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': '', 'Structural': '', 'ConfidenceI...",FHRS,"{'longitude': '0.08384000', 'latitude': 51.490...",,4623.972328,True


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [43]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query = {}
results = establishments.find(query)
# Use count_documents to display the number of documents in the result
establishments.count_documents(query)


query = {'LocalAuthorityName':{'$regex':'London'},'RatingValue': {'$gte':'4'}}


# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

0
None


In [49]:
# Convert the result to a Pandas DataFrame
london = (establishments.find(query))
london_df = pd.DataFrame(london)

# Display the number of rows in the DataFrame
pprint(london_df.shape[0])
# Display the first 10 rows of the DataFrame
london_df.head(10)

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [45]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query ={'geocode.latitude':{'$gte':latitude-degree_search,'$lte':latitude+degree_search},
        'geocode.longitude':{'$gte':longitude-degree_search,'$lte':longitude+degree_search},
        'RatingValue':'5'}

sort =[('scores.Hygiene', 1)]
limit = 5

# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))

[]


In [46]:
# Convert result to Pandas DataFrame
establishments_1 = list(establishments.find(query).sort(sort).limit(limit))
df_1 = pd.DataFrame(establishments_1)
print(df_1)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [47]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
bad_hygiene = {'$match': {'scores.Hygiene':0}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': {"LocalAuthorityName": "$LocalAuthorityName",
                          'count': { '$sum': 1 }
                         }
              }}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'count': +1 }}
# Print the number of documents in the result
pipeline = [bad_hygiene, group_query, sort_values]
# Print the first 10 results
results = list(establishments.aggregate(pipeline))
# pprint("Number of documents in result: ", len(results))
pprint(results[0:10])


[]


In [48]:
# Convert the result to a Pandas DataFrame
bad_hygiene_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print("Total rows in bad_hygiene_df is ",len(bad_hygiene_df))
# Display the first 10 rows of the DataFrame
bad_hygiene_df.head(10)

Total rows in bad_hygiene_df is  0


""
